In [1]:
from __future__ import print_function
import glob, os, gc, sys
import os.path
import csv
import numpy as np
np.random.seed(1337)  # for reproducibility
from time import time
from subprocess import (call, Popen, PIPE)
from itertools import product
from IPython.display import display
from PIL import Image
from IPython.display import Image as IPImage
import shutil
import re
import xml.etree.ElementTree as ET
import time



##Path to Data
basepath = "/home/ubuntu/efs/SLAV_Data/TrainingData/" 

In [ ]:
#Test basic model on SLAV

import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import np_utils
from keras import backend as K
from keras.utils import multi_gpu_model

config = tf.ConfigProto(log_device_placement=True)
config.gpu_options.allocator_type = 'BFC'
config.gpu_options.allow_growth = True
sess = tf.Session(config = config)
K.set_session(sess)
# dimensions of our images.
img_width, img_height = 64,64

train_data_dir = os.path.join(basepath, 'Train')
validation_data_dir = os.path.join(basepath, 'Validate')
nb_train_samples = 5393
nb_validation_samples = 400
epochs = 100
batch_size = 8


if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.3))
model.add(Dense(4))
model.add(Activation('sigmoid'))

model = multi_gpu_model(model,4)

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.summary()
# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(rescale=1./255)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(64, 64),
    color_mode = 'rgb',
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(64, 64),
    color_mode = 'rgb',
    batch_size=batch_size,
    class_mode='categorical')

model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // (batch_size * 4),
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // (batch_size * 4), verbose=1)

model.save_weights('first_try_SLAV.h5')

Using TensorFlow backend.


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
conv2d_1_input (InputLayer)     (None, 64, 64, 3)    0                                            
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 64, 64, 3)    0           conv2d_1_input[0][0]             
__________________________________________________________________________________________________
lambda_2 (Lambda)               (None, 64, 64, 3)    0           conv2d_1_input[0][0]             
__________________________________________________________________________________________________
lambda_3 (Lambda)               (None, 64, 64, 3)    0           conv2d_1_input[0][0]             
__________________________________________________________________________________________________
lambda_4 (

In [ ]:
scoreSeg = parallel_model.evaluate_generator(validation_generator,10000, workers=8, use_multiprocessing=True)
print("Accuracy = ",scoreSeg[1])


In [ ]:
predict = parallel_model.predict_generator(validation_generator,10000, workers=8, use_multiprocessing=True)

In [ ]:
model.save('MNIST_model.h5')

In [ ]:
import matplotlib.pyplot as plt
from keras.utils import plot_model
from keras.models import load_model
model = load_model('MNIST_model.h5')
plot_model(model, to_file='model.png')